# Making use of Matrix structure

The NAG library for Python often contains specialised varations for any given mathematical problem.  In the case of linear solvers, one set of variations caters for coefficient matrices that have a certain structure.  When your problem exhibits such structure, it often pays divideds to use the specialised solver.

## Solving linear systems of Toeplitz matrices - An example of exploiting stucture

Toeplitz matrices occur in various applications. A couple of examples are given at http://jack.valmadre.net/notes/2015/03/28/symmetric-positive-toeplitz/

Here we solve a linear system where the coefficient matrix is a Toeplitz matrix.  First, we make no use of the underlying structure and just use NAG's **dgesv**

In [1]:
from naginterfaces.library.lapacklin import dgesv   # A general solver
from naginterfaces.library.linsys import real_toeplitz_solve # A toeplitz solver
from pytictoc import TicToc
import numpy as np
import scipy.linalg

timer = TicToc()

# Construct a real, symmetric, positive definite toeplitz matrix 
matrix_size = 5000
t = np.arange(0,matrix_size);
a = np.exp(-np.abs(t)/10);
# The toeplitz matrix is defined by it's diagonals.  We can construct the full matrix from the diagonals using scipy
A = scipy.linalg.toeplitz(a, a)
# Construct and a random Righ hand side
np.random.seed(2)
b = np.random.rand(matrix_size,1)

timer.tic()
[asol,ipiv,x_nag_gen] = dgesv(A, b)
timer.toc()

Elapsed time is 5.394560 seconds.


We will now solve the same problem but make use of the function **real_toeplitz_solve**

In [2]:
# NAG's toeplitz solver requires that b be of dimension N instead of N,1
# So we reconstruct it with the correct dimension
np.random.seed(2)
b = np.random.rand(matrix_size)

timer.tic()
[x_nag_toe,p] = real_toeplitz_solve(matrix_size,a,b,wantp=False)
timer.toc()

Elapsed time is 0.035298 seconds.


In [3]:
# Let's see how many times faster it is to use the toeplitz solver
5.39/0.035

153.99999999999997

It is **well over 100x faster** to use the Toeplitz solver.  We should check that we get the same answers?  
Annoyingly, the solution matrices are are different shapes!

In [5]:
#Solution from the general solver
x_nag_gen.shape

(5000, 1)

In [6]:
#Solution from the toeplitz solver
x_nag_toe.shape

(5000,)

but the values agree to high accuracy

In [8]:
np.max(abs(x_nag_gen - np.reshape(x_nag_toe,(matrix_size,1))))

1.1191048088221578e-13

## Other linear solvers in the NAG library for Python that make use of various matrix structures 

Your application may not be able to make use of Toeplitz solvers but it may well be able to take advantage of one of the other specialised linear solvers in the NAG library.  Here's the full list